In [19]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
import torch
import torch.nn as nn
from contextlib import nullcontext
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [20]:
class TrainArgs():
    #文件管理
    out_dir = './pretrain_output'
    checkpoint_path = "./pretrain_output/latest_checkpoint.pth"
    data_path = 'data/pretrain_hq.jsonl'
    #神经网络训练管理
    epochs = 2
    batch_size = 32
    accumulation_steps = 4
    learning_rate = 5e-4
    warm_up = 0
    grad_clip = 1
    dtype = 'bfloat16'
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    num_workers = min(8, os.cpu_count()) 
    log_interval = 50
    save_interval = 2000
    ctx = nullcontext() if device == "cpu" else torch.cuda.amp.autocast()

/tmp/ipykernel_30403/2837662099.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  ctx = nullcontext() if device == "cpu" else torch.cuda.amp.autocast()


In [2]:
class EvalArgs():
    hidden_size = 512
    temperature = 0.85
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_hidden_layers = 8
    history_cnt = 0
    model_mode = 0 #0: 预训练模型，1: SFT-Chat模型，2: RLHF-Chat模型，3: Reason模型，4: RLAIF-Chat模型"
    max_seq_len = 8192
    top_p =0.85

初始化模型

In [21]:
def init_model(args):
    tokenizer = AutoTokenizer.from_pretrained('./model/')
    
        
    modes = {0: 'pretrain', 1: 'full_sft', 2: 'rlhf', 3: 'reason', 4: 'grpo'}
    ckp = f'{modes[args.model_mode]}_output/latest_checkpoint.pth'

    model = MiniMindForCausalLM(MiniMindConfig(
        hidden_size=args.hidden_size,
        num_hidden_layers=args.num_hidden_layers,
        
    ))

    # 1. 加载完整的 checkpoint 字典，并指明 weights_only=False
    print(f"Loading checkpoint from {ckp}...")
    checkpoint = torch.load(ckp, map_location=args.device, weights_only=False)
    
    # 2. 从字典中提取模型的 state_dict 来加载
    model.load_state_dict(checkpoint['model'], strict=True)
    
    print("Checkpoint loaded successfully.")
    return model.eval().to(args.device), tokenizer
    

In [4]:
def get_prompt_datas(args):
    if args.model_mode == 0:
        # pretrain模型的接龙能力（无法对话）
        prompt_datas = [
            '马克思主义基本原理',
            '人类大脑的主要功能',
            '万有引力原理是',
            '世界上最高的山峰是',
            '二氧化碳在空气中',
            '地球上最大的动物有',
            '杭州市的美食有'
        ]
    else:
       
        # 通用对话问题
        prompt_datas = [
            '请介绍一下自己。',
            '你更擅长哪一个学科？',
            '鲁迅的《狂人日记》是如何批判封建礼教的？',
            '我咳嗽已经持续了两周，需要去医院检查吗？',
            '详细的介绍光速的物理概念。',
            '推荐一些杭州的特色美食吧。',
            '请为我讲解“大语言模型”这个概念。',
            '如何理解ChatGPT？',
            'Introduce the history of the United States, please.'
        ]
        
            

    return prompt_datas

正式评估

In [22]:
model, tokenizer = init_model(EvalArgs)

prompts = get_prompt_datas(EvalArgs)
test_mode = int(input('[0] 自动测试\n[1] 手动输入\n'))
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

messages = []
for idx, prompt in enumerate(prompts if test_mode == 0 else iter(lambda: input('👶: '), '')):
    
    if test_mode == 0: print(f'👶: {prompt}')

    messages = messages[-EvalArgs.history_cnt:] if EvalArgs.history_cnt else []
    messages.append({"role": "user", "content": prompt})

    new_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    ) if EvalArgs.model_mode != 0 else (tokenizer.bos_token + prompt)

    inputs = tokenizer(
        new_prompt,
        return_tensors="pt",
        truncation=True
    ).to(EvalArgs.device)

    print('🤖️: ', end='')
    generated_ids = model.generate(
        inputs["input_ids"],
        max_new_tokens=EvalArgs.max_seq_len,
        num_return_sequences=1,
        do_sample=True,
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
        top_p=EvalArgs.top_p,
        temperature=EvalArgs.temperature
    )

    response = tokenizer.decode(generated_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    messages.append({"role": "assistant", "content": response})
    print('\n\n')


Loading checkpoint from pretrain_output/latest_checkpoint.pth...
Checkpoint loaded successfully.
👶: 马克思主义基本原理
🤖️: 是什么？马克思主义基本原理是将人类智慧和创造力集中在人类的思维能力之外，即通过科学方法和思维手段，在不同的历史时期或文化背景下，将人类智慧与创造力相结合，从而实现人类文明的进步和发展。



👶: 人类大脑的主要功能
🤖️: 是什么？ 人类大脑主要功能是通过神经元之间的连接来进行信息处理和信息传递。在大脑中，神经元通过神经元之间的连接产生信息，这些信息传递给大脑。神经元通过这些神经元连接成神经元，这些神经元之间通过电信号传递信息。神经元是神经系统的重要组成部分，它们通过神经元之间的信号传递，并传递到大脑的各个部位。神经元之间的连接可以是感知、运动、学习、记忆等多种功能，神经元之间的相互作用可以影响认知和行为。



👶: 万有引力原理是
🤖️: 如何运作的？万有引力原理是通过引力作用力来描述的。

引力是由质量和距离之间产生的，这些距离与距离的平方成反比。

当两个物体之间存在引力时，它们之间的引力会发生变化，这个变化的结果就是它们之间相互吸引的关系。引力会将物体吸引到它们的轨道，这会影响它们之间的引力。

当两个物体之间存在引力时，它们的引力会发生变化，例如，当两个物体受到一个力时，它们之间的引力会改变它们之间的距离。如果两个物体之间的引力不同，它们之间的引力也会产生变化，例如，当两个物体的质量增加时，它们之间的引力会减少。



👶: 世界上最高的山峰是
🤖️: 珠穆朗玛峰，海拔8,848米。它位于喜马拉雅山脉中，被称为“地球之巅”，是地球上最高的山峰。珠穆朗玛峰是世界上最高的山峰，海拔8,848米，位于喜马拉雅山脉中，是地球上最高的山峰。



👶: 二氧化碳在空气中
🤖️: 以减少其二氧化碳含量的方式生成氧气。二氧化碳的化学式为H2O，其中CO2和CO2在空气中以减少其二氧化碳含量的方式生成氧气。
在空气中，二氧化碳的含量减少了，导致了二氧化碳的含量增加。CO2和CO2在空气中以减少其二氧化碳含量的方式生成氧气。



👶: 地球上最大的动物有
🤖️: 多少个？地球上最大的动物是鲸鱼，它们有52个头，最长的可达72公里。鲸鱼是地球上最